# Comparison To Simulation

We use OpenRocket to simulate the launch.

In [ ]:
from IPython.display import Image
Image(filename='../../simulation/openrocket_screenshot.png')

In [ ]:
from numpy import loadtxt, array, subtract, divide, multiply, median, std, var, abs
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')

g_0 = 9.80665
t_0 = 117853569585227


# Sim
columns = loadtxt("../../simulation/Launch-12.csv", delimiter=',', unpack=True)
or_time = columns[0]
or_alt = columns[1]
or_accel = columns[3]


# ADIS
columns = loadtxt("../fc-data/ADIS.csv", delimiter=',', unpack=True)
adis_time = divide(subtract(columns[1], t_0), 1e9)
acc_x, acc_y, acc_z = columns[6:9]

acc_x = subtract(acc_x, g_0)
# lightly lowpass
from scipy.signal import butter, lfilter, freqz
# Filter requirements.
order = 6
fs = 819.2       # sample rate, Hz
cutoff = 20   # desired cutoff frequency of the filter, Hz
nyq = 0.5 * fs
normal_cutoff = cutoff / nyq
# Get the filter coefficients so we can check its frequency response.
b, a = butter(order, normal_cutoff, btype='low', analog=False)
acc_x_filter = lfilter(b, a, acc_x)

# TeleMetrum
columns = loadtxt("../TeleMetrum/2015-07-19-serial-1372-flight-0002.csv",
                  delimiter=',', comments='#',unpack=True,
                  usecols=(4,5,6,10,11,13,16,21,22,25))
tele_time = columns[0]
clock = columns[1]
rssi = columns[2]
tele_accel = columns[3]
tele_pressure = columns[4]
tele_alt = columns[5]
tele_temp = columns[6]
tele_gpslock, tele_gpsnsat = columns[7:9]
tele_gpsalt = columns[9]

# ARTS2
FEET2METERS = 0.3048
columns = loadtxt("../ARTS2/2015-07-19_ARTS2_Flight_Data_Interpreted_Data.csv",
                  delimiter=',', comments='#',unpack=True)
arts_time = columns[0]
arts_accel = multiply(columns[1], g_0)
arts_vel = columns[2]
arts_ialt = columns[3]
arts_palt = multiply(columns[4], FEET2METERS)

# ADIS Altitude
columns = loadtxt("uncalibrated_integrated_altitude.csv", delimiter=',', unpack=True)
alt_time = columns[0]
imu_alt   = columns[1]

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"Simulation Acceleration vs Meausred Acceleration")
plt.ylabel(r"Acceleration [m/s${}^2$]")
plt.xlabel(r"Mission Elapsed Time [s]")


plt.plot(adis_time[100:-100], acc_x_filter[100:-100], lw=1, alpha=0.75,  label="ADIS IMU Acceleration")
plt.plot(tele_time, tele_accel, lw=1, alpha=0.75,  label="TeleMetrum Acceleration")
plt.plot(arts_time, arts_accel, lw=1, alpha=0.75,  label="ARTS2 Acceleration")
plt.plot(or_time, or_accel, 'k-', alpha=0.4,  lw=2, label="OpenRocket Vertical Acceleration")

plt.xlim([-5,42])
ax1.set_ylim([-50,120])
ax1.legend(loc=1)
plt.show()

## Transonic

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"Simulation Acceleration vs Measured Acceleration Near Supersonic")
plt.ylabel(r"Acceleration [m/s${}^2$]")
plt.xlabel(r"Mission Elapsed Time [s]")

# ADIS Altitude
columns = loadtxt("uncalibrated_integrated_velocity.csv", delimiter=',', unpack=True)
vel_time = columns[0]
imu_vel   = columns[1]

maxidx = (abs(imu_vel-max(imu_vel))).argmin()

begin_super_idx = (abs(imu_vel[0:maxidx]-330)).argmin()
end_super_idx = (abs(imu_vel[maxidx:]-330)).argmin() + maxidx

plt.fill_between([vel_time[begin_super_idx], vel_time[end_super_idx]], -200, [200,200], color='red', alpha=0.11, label="Supersonic")

plt.plot(adis_time[100:-100], acc_x_filter[100:-100], lw=1, alpha=0.75,  label="ADIS IMU Acceleration")
plt.plot(tele_time, tele_accel, lw=1, alpha=0.75,  label="TeleMetrum Acceleration")
plt.plot(arts_time, arts_accel, lw=1, alpha=0.75,  label="ARTS2 Acceleration")

plt.plot(or_time, or_accel, 'k-', alpha=0.4,  lw=2, label="OpenRocket Vertical Acceleration")

plt.xlim([2,9])
ax1.set_ylim([-50,100])
ax1.legend(loc=1)
plt.show()

## Altitude

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,6))
plt.title(r"Simulation Altitude vs Measured Altitude")
plt.ylabel(r"Altitude AGL [km]")
plt.xlabel(r"Mission Elapsed Time [s]")

plt.plot(alt_time, imu_alt, lw=1, alpha=0.75,  label="ADIS IMU Altitude")
plt.plot(tele_time, tele_alt, lw=1, alpha=0.75,  label="TeleMetrum Altitude (Pressure)")
#plt.plot(arts_time, arts_accel, lw=1, alpha=0.75,  label="ARTS2 Acceleration")
plt.plot(or_time, or_alt, 'k-', alpha=0.4,  lw=2, label="OpenRocket Altitude")

plt.xlim([-5,42])
#ax1.set_ylim([-50,120])
ax1.legend(loc=2)
plt.show()